## Features extraction

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib.colors import ListedColormap
import plotly.express as px
import subprocess
import os

In [ ]:
LABELS = [
    "get, take",
    "place, place-on, place-back, put, put-back",
    "open",
    "close",
    "clean, wipe-off, wipe, wash, rinse",
    "cut, chop, slice",
    "mix, mix-around, stir, whisk",
    "pour"
]

In [ ]:
if False:   

    # Assicurati di essere nella directory corretta
    os.chdir("C:/Users/iplov/Desktop/Advanced ML/Progetto/github andrew/Egocentric-Vision")
    # Comando da eseguire
    command = [
        'python', 'save_feat.py',
        'config=configs/I3D_save_feat.yaml',
        'dataset.shift=D1-D1',
        'dataset.RGB.data_path=D:/ek_data/frames',
        'save.dense_sampling.RGB=true',
        'save.num_frames_per_clip.RGB=5',
        'name=saved_feat_I3D_5_Dense'
    ]

    # Esegui il comando
    result = subprocess.run(command, capture_output=True, text=True)

    # Stampa l'output e l'errore
    print('Output:', result.stdout)
    #print('Error:', result.stderr)

    # Verifica il codice di uscita
    if result.returncode != 0:
        print(f'Command failed with return code {result.returncode}')

In [ ]:
# Assicurati di essere nella directory corretta
os.chdir("C:/Users/iplov/Desktop/Advanced ML/Progetto/github andrew/Egocentric-Vision")
num_frames_list = [5,10,25]
#DENSE
if False:
    for n in num_frames_list:
        # Comando da eseguire
        command = [
            'python', 'save_feat.py',
            'config=configs/I3D_save_feat.yaml',
            'dataset.shift=D1-D1',
            'dataset.RGB.data_path=D:/ek_data/frames',
            'save.dense_sampling.RGB=true',
            f'save.num_frames_per_clip.RGB={n}',
            f'name=saved_feat_I3D_{n}_dense'
        ]

        # Esegui il comando
        result = subprocess.run(command, capture_output=True, text=True)

        # Stampa l'output e l'errore
        print('Output:', result.stdout)
        #print('Error:', result.stderr)

        # Verifica il codice di uscita
        if result.returncode != 0:
            print(f'Command failed with return code {result.returncode}')

    #UNIFORM
    for n in num_frames_list:
        print(f"UNIFORM n = {n}")
        # Comando da eseguire
        command = [
            'python', 'save_feat.py',
            'config=configs/I3D_save_feat.yaml',
            'dataset.shift=D1-D1',
            'dataset.RGB.data_path=D:/ek_data/frames',
            'save.dense_sampling.RGB=false',
            f'save.num_frames_per_clip.RGB={n}',
            f'name=saved_feat_I3D_{n}_uniform'
        ]

        # Esegui il comando
        result = subprocess.run(command, capture_output=True, text=True)

        # Stampa l'output e l'errore
        print('Output:', result.stdout)
        #print('Error:', result.stderr)

        # Verifica il codice di uscita
        if result.returncode != 0:
            print(f'Command failed with return code {result.returncode}')

## Read extracted feature

In [ ]:
features = pd.read_pickle("./saved_features/saved_feat_I3D_10_dense_D1_test.pkl")
#print(features['features'][0])
print(len(features['features']))
#print(features['features'][0]['features_RGB'])
#print(features['features'][0]['label'])
print(features['features'][0].keys())
list_of_features = [np.mean(np.array(feature['features_RGB']),axis=0) for feature in features["features"]]
clips_feature = list_of_features
labels = [feature['label'] for feature in features["features"]]
labels_extended = [label for label in labels for _ in range(5)]
print(labels_extended[0:20])
list_of_features = [np.array(f) for feature in features["features"] for f in feature['features_RGB']]

# K-Means

In [ ]:
clips_feature_tensor = np.array(clips_feature)

print(clips_feature_tensor.shape)

## plot using TSNE 2D

In [ ]:
#plot k-means labels
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(clips_feature_tensor)
cmap = ListedColormap(['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black', 'olive'])

y_kmeans = kmeans.predict(clips_feature_tensor)

tsne = TSNE(n_components=2, random_state=42)
# Addestrare il modello e trasformare i dati
X = tsne.fit_transform(clips_feature_tensor)
# scatter plot of X values
plt.figure(1)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans,cmap=cmap, s=50)

plt.show()


#plot real labels
cmap = ListedColormap(['gray', 'orange', 'black', 'red', 'green', 'olive', 'purple', 'blue'])
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cmap(i),  markersize=10, label=desc) 
                for i, desc in enumerate(LABELS)]
plt.figure(2)

# scatter plot of X values
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap=cmap, s=50)
# Add the legend
plt.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1))


plt.show()


## plot using TSNE 3D

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(clips_feature_tensor)
cmap = ['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black', 'olive']
kmeans_color_map = {f"{i}": color for i, color in enumerate(cmap)}

y_kmeans = kmeans.predict(clips_feature_tensor)

tsne = TSNE(n_components=3, random_state=42)
# Addestrare il modello e trasformare i dati
X = tsne.fit_transform(clips_feature_tensor)
# scatter plot of X values
df = pd.DataFrame(X, columns=['x', 'y', 'z'])
df['kmeans'] = y_kmeans.astype(str)
df['labels'] = np.array(labels).astype(str)


#plot label k_means
fig = px.scatter_3d(df, x='x', y='y', z='z', color="kmeans", color_discrete_map=kmeans_color_map, title="plot using k_means label")
fig.update_traces(marker_size=4)
fig.show()

cmap = ['blue', 'orange', 'black', 'red', 'green', 'gray', 'purple', 'olive']
kmeans_color_map = {f"{i}": color for i, color in enumerate(cmap)}
custom_legend = {f"{i}": label_name for i, label_name in enumerate(LABELS)}
#plot real label
fig = px.scatter_3d(df, x='x', y='y', z='z', color="labels", color_discrete_map=kmeans_color_map, title="plot using real label",labels=custom_legend)
fig.update_traces(marker_size=4)
fig.show()

## TRAINING

In [9]:

command = [
    'python', 'train_finalClassifier.py',
    'name=final_classifier',
    'logname=I3D_10_dense_D1',
    'action=train',
]

# Esegui il comando
result = subprocess.run(command, capture_output=True, text=True)

# Stampa l'output e l'errore
print('Output:', result.stdout)
print('Error:', result.stderr)

# Verifica il codice di uscita
if result.returncode != 0:
    print(f'Command failed with return code {result.returncode}')

Output: 
Error: /Users/andreavannozzi/GithubProjects/Egocentric-Vision/env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/andreavannozzi/GithubProjects/Egocentric-Vision/env/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <0EB69795-4559-3C98-9EA1-35B6A988BB99> /Users/andreavannozzi/GithubProjects/Egocentric-Vision/env/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <E4E2FFCA-031E-3974-A7B0-45408D7F4956> /Users/andreavannozzi/GithubProjects/Egocentric-Vision/env/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")
--- Logging error ---
Traceback (most recent call last):
  File "/Users/andreavannozzi/GithubProjects/Egocentric-Vision/env/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/Users/andreavannozzi/GithubPro